# Data Cleaning
convert census csv files into usable data

housingdf, labordf, geodf, popdf

In [ ]:
import pandas as pd

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi")


In [210]:
# Function to clean and convert columns
def clean_and_convert(column):
    return pd.to_numeric(
        column.replace({",": "", "%": ""}, regex=True), errors="coerce"
    )

## Housing Units (housingdf)

In [237]:
housingdf = pd.read_csv("data/raw/MSA_HOUSING_UNITS.csv")
housingdf = housingdf.rename(
    columns={
        "Label (Grouping)": "msa",
        "Total:": "Total_Homes",
        "Total:!!Occupied": "Occupied_Homes",
        "Total:!!Vacant": "Vacant_Homes",
    }
)

housingdf.dropna(inplace=True)

# List of columns to clean
columns_to_clean = [
    "Total_Homes",
    "Occupied_Homes",
    "Vacant_Homes",
]

housingdf['msa'] = housingdf['msa'].str.strip()


# # Apply the function only to specified columns
housingdf[columns_to_clean] = housingdf[columns_to_clean].apply(clean_and_convert)

housingdf['Vacant_Homes_Percent'] = ''

for msa in housingdf['msa']:
    housingdf.loc[housingdf['msa'] == msa, "Vacant_Homes_Percent"] = (housingdf.loc[housingdf['msa'] == msa, "Vacant_Homes"]/housingdf.loc[housingdf['msa'] == msa, "Total_Homes"]) * 100

housingdf["Vacant_Homes_Percent"] = pd.to_numeric(housingdf['Vacant_Homes_Percent'])
housingdf.head()

,msa,Total_Homes,Occupied_Homes,Vacant_Homes,Vacant_Homes_Percent
1,"Aberdeen, SD Micro Area",19825,17676,2149,10.839849
3,"Aberdeen, WA Micro Area",36058,29869,6189,17.164014
5,"Abilene, TX Metro Area",74341,65806,8535,11.480879
7,"Ada, OK Micro Area",17406,15179,2227,12.794439
9,"Adrian, MI Micro Area",43577,38581,4996,11.464764


## Labor Info (labordf)

In [15]:
# "Label (Grouping)","EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate","COMMUTING TO WORK!!Workers 16 years and over!!Worked from home","COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)","INCOME AND BENEFITS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)","INCOME AND BENEFITS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Median earnings for workers (dollars)"

labordf = pd.read_csv("data/raw/MSA_LABOR_TRAVEL_INCOME.csv")
labordf = labordf.rename(
    columns={
        "Label (Grouping)": "msa",
        "EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate": "Unemployment",
        "COMMUTING TO WORK!!Workers 16 years and over!!Worked from home": "Remote",
        "COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)": "Travel_Time",
        "INCOME AND BENEFITS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)": "Household_Income",
        "INCOME AND BENEFITS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Median earnings for workers (dollars)": "Worker_Income",
    }
)
msa = labordf["msa"].iloc[::3].tolist()  # row
Unemployment = labordf["Unemployment"].iloc[2::3].tolist()  # percent
Remote = labordf["Remote"].iloc[2::3].tolist()  # percent
Travel_Time = labordf["Travel_Time"].iloc[1::3].tolist()  # value
Household_Income = labordf["Household_Income"].iloc[1::3].tolist()  # value
Worker_Income = labordf["Worker_Income"].iloc[1::3].tolist()  # value

cleandf = pd.DataFrame(
    {
        "msa": msa,
        "Unemployment (%)": Unemployment,
        "Remote (%)": Remote,
        "Travel_Time": Travel_Time,
        "Household_Income": Household_Income,
        "Worker_Income": Worker_Income,
    }
)

# List of columns to clean
columns_to_clean = [
    "Unemployment (%)",
    "Remote (%)",
    "Travel_Time",
    "Household_Income",
    "Worker_Income",
]

# Apply the function only to specified columns
cleandf[columns_to_clean] = cleandf[columns_to_clean].apply(clean_and_convert)

labordf = cleandf

labordf.head()

,msa,Unemployment (%),Remote (%),Travel_Time,Household_Income,Worker_Income
0,"Aberdeen, SD Micro Area",3.5,6.0,13.9,70693,40136
1,"Aberdeen, WA Micro Area",6.4,8.4,24.0,59105,35642
2,"Abilene, TX Metro Area",2.8,7.0,18.2,61924,33841
3,"Ada, OK Micro Area",4.3,3.2,17.9,59457,39638
4,"Adrian, MI Micro Area",5.6,7.3,26.8,65484,35277


## Geographic Info (geodf)

In [ ]:
# geodf = pd.read_csv("data/raw/MSA_REFERENCE.csv", encoding="unicode_escape")

# geodf = geodf.drop_duplicates(subset=['msa'])

# geodf['latitude'] = ''
# geodf['longitude'] = ''
# geodf.head()

# # geodf[geodf['county_name'] == "Brown County, South Dakota"]['latitude'] = 50
# geodf.loc[geodf['county_name'] == "Brown County, South Dakota", "latitude"] = 50
# geodf.head()

In [ ]:
# geodf = pd.read_csv("data/raw/geodf_new.csv", encoding="unicode_escape")

# def geocode_county(county_name):
#     location = geolocator.geocode(county_name)
#     return[location.latitude, location.longitude]

# for county_name in geodf['county_name']:
#     geodf.loc[geodf['county_name'] == county_name, "latitude"] = geocode_county(county_name)[0]
#     geodf.loc[geodf['county_name'] == county_name, "longitude"] = geocode_county(county_name)[1]

# geodf.head()
# geodf.to_csv('data/raw/geodf_new.csv', index=False) 

## Population data (popdf)

In [ ]:
popdf = pd.read_csv('data/raw/MSA_POP_HOUSING.csv')
popdf = popdf.rename(
    columns={
        "Label (Grouping)": "msa",
        "SEX AND AGE!!Total population": "Population",
        "Total housing units": "Housing",
    }
)

popdf.drop("Housing", axis=1)
msa = popdf["msa"].iloc[::3].tolist()  # row
Population = popdf["Population"].iloc[1::3].tolist()  # value
cleandf = pd.DataFrame(
    {
        "msa": msa,
        "Population": Population,
    }
)

# List of columns to clean
columns_to_clean = [
    "Population"
]

# Apply the function only to specified columns
cleandf[columns_to_clean] = cleandf[columns_to_clean].apply(clean_and_convert)

popdf = cleandf

popdf.head()

## Export
housingdf, labordf, geodf, popdf

In [239]:
housingdf.to_csv('data/clean/housing.csv', index=False)
# labordf.to_csv('data/clean/labor.csv', index=False)
# popdf.to_csv('data/clean/population.csv', index=False)

### Combine into 1 Static Dataset

In [243]:
housingdf = pd.DataFrame(pd.read_csv("data/clean/housing.csv"))
labordf = pd.DataFrame(pd.read_csv("data/clean/labor.csv"))
popdf = pd.DataFrame(pd.read_csv("data/clean/population.csv"))

In [244]:
geodf = pd.DataFrame(pd.read_csv("data/clean/geodf.csv"))
geodf = geodf[['msa','msa_name', 'latitude', 'longitude']] 

geodf = geodf.rename(columns={"msa": "msa_id"})
geodf = geodf.rename(columns={"msa_name": "msa"})

In [245]:
maindf = popdf.merge(labordf, on="msa", how="right")
maindf = maindf.merge(geodf, on="msa")
maindf = maindf.merge(housingdf, on="msa")
maindf.head()

,msa,Population,Unemployment (%),Remote (%),Travel_Time,Household_Income,Worker_Income,msa_id,latitude,longitude,Total_Homes,Occupied_Homes,Vacant_Homes,Vacant_Homes_Percent
0,"Aberdeen, SD Micro Area",42292,3.5,6.0,13.9,70693,40136,10100,45.583297,-98.367479,19825,17676,2149,10.839849
1,"Aberdeen, WA Micro Area",75672,6.4,8.4,24.0,59105,35642,10140,47.174417,-123.865883,36058,29869,6189,17.164014
2,"Abilene, TX Metro Area",176656,2.8,7.0,18.2,61924,33841,10180,32.281041,-99.361244,74341,65806,8535,11.480879
3,"Ada, OK Micro Area",38116,4.3,3.2,17.9,59457,39638,10220,34.707331,-96.681485,17406,15179,2227,12.794439
4,"Adrian, MI Micro Area",99263,5.6,7.3,26.8,65484,35277,10300,41.873526,-84.062432,43577,38581,4996,11.464764


In [246]:
maindf.describe()

,Population,Unemployment (%),Remote (%),Travel_Time,Household_Income,Worker_Income,msa_id,latitude,longitude,Total_Homes,Occupied_Homes,Vacant_Homes,Vacant_Homes_Percent
count,7.920000e+02,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,792.000000,7.920000e+02,7.920000e+02,792.000000,792.000000
mean,2.782825e+05,5.115530,7.409848,22.368939,63306.179293,36709.092172,29879.364899,38.224235,-92.807558,1.172822e+05,1.067122e+05,10570.075758,12.207993
std,8.150493e+05,1.710228,3.499031,3.738046,13170.933114,5681.931963,11417.981744,5.144006,13.657454,3.194243e+05,2.989367e+05,21913.044901,6.417926
min,1.279500e+04,0.400000,1.300000,12.900000,33658.000000,12955.000000,10100.000000,21.468151,-159.494511,5.836000e+03,4.536000e+03,487.000000,3.495686
25%,3.947700e+04,3.900000,5.000000,19.900000,55000.000000,33629.250000,20090.000000,34.473751,-98.387737,1.835950e+04,1.572850e+04,2153.250000,8.187112
50%,6.940750e+04,5.000000,6.700000,22.400000,61924.000000,36168.500000,29960.000000,38.826096,-89.636018,3.273700e+04,2.727700e+04,4063.000000,10.788462
75%,1.819272e+05,6.000000,9.100000,24.700000,69369.500000,39639.500000,39870.000000,41.905726,-83.356710,7.878100e+04,7.007475e+04,8644.750000,13.794706
max,1.311192e+07,13.200000,24.400000,35.300000,151713.000000,83135.000000,49820.000000,61.216313,-68.595777,4.721766e+06,4.494733e+06,227033.000000,58.716738


In [247]:
maindf.to_csv('data/clean/combined.csv', index=False)